In [3]:
pip install requests openpyxl

In [4]:
import requests
import json
import time
import openpyxl

In [5]:
def search_app_id(app_name):
    url = f"https://itunes.apple.com/search?term={app_name}&entity=software"
    response = requests.get(url)
    data = response.json()
    results = data.get('results', [])
    for result in results:
        name = result.get('trackName')
        app_id = result.get('trackId')
        print(f"Name: {name}, ID: {app_id}")
    return results

In [6]:
# Test
results = search_app_id("day-one-journal-private-diary")
if results:
    app_id = results[0]['trackId']
    print(f"App ID: {app_id}")

Name: Day One Journal: Private Diary, ID: 1044867788
Name: 1 Second Everyday Diary, ID: 587823548
Name: Tinybeans Private Family Album, ID: 521633042
Name: Diarly: Diary, Private Journal, ID: 1387167765
Name: Diary-Journal with lock, ID: 1590102229
Name: Happyfeed: Video Diary Journal, ID: 707073943
Name: Diary With Password, ID: 1252208166
Name: May Diary - Gratitude Journal, ID: 658236285
Name: My Diary - Private Journal, ID: 6478656385
Name: Diary - Private Note With Lock, ID: 1454490013
Name: My Wonderful Days Journal, ID: 434356065
Name: Personal Diary (Journal) App, ID: 1052570257
Name: OneDay - Diary Journal, ID: 1578156802
Name: Diary, Journal, Notes - Diaro, ID: 882519460
Name: GoJournal: Diary & Planner, ID: 1596865689
Name: Video Dairy: Daily Journal, ID: 1483531474
Name: Secret Diary-Gratitude Journal, ID: 1107242504
Name: Simple Diary - Daily Journal, ID: 1388195923
Name: Diary Journal - With Lock, ID: 1572225395
Name: Orange Diary-Gratitude Journal, ID: 1066756614
Name: J

In [16]:
def save_content(app_id, workbook, worksheet, total_reviews):

    row = 2  # skip title
    reviews_fetched = 0
    page = 1

    while reviews_fetched < total_reviews:
      for page in range(1, 11):  # first 10 page
          url = f"https://itunes.apple.com/rss/customerreviews/page={page}/id={app_id}/sortby=mostrecent/json?l=en&&cc=us" #cc = country name
          response = requests.get(url)
          if response.status_code == 200:
              try:
                  data = response.json().get("feed", {}).get("entry", [])
                  if not data:
                    print("No more reviews found.")
                    break

                  for entry in data:
                      if reviews_fetched >= total_reviews:
                        break
                      name = entry.get('author', {}).get('name', {}).get('label')
                      user_id = entry.get('id', {}).get('label')
                      review_date = entry.get('updated', {}).get('label')
                      rating = entry.get('im:rating', {}).get('label')
                      content = entry.get('content', {}).get('label')

                      # import to excel
                      worksheet.cell(row=row, column=1, value=name)
                      worksheet.cell(row=row, column=2, value=user_id)
                      worksheet.cell(row=row, column=3, value=review_date)
                      worksheet.cell(row=row, column=4, value=rating)
                      worksheet.cell(row=row, column=5, value=content)
                      row += 1
                      reviews_fetched += 1

                  page += 1

              except KeyError:
                  print("No more reviews")
                  break
          else:
              break
          time.sleep(2)  # lag for 2 secs

def main():
    app_name = input("APP's name:\n")
    results = search_app_id(app_name)
    if not results:
        print("No related APP")
        return

    app_id = results[0].get('trackId')  # Use the first App ID showed


    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    valid_title = app_name.replace(':', '').replace('\\', '').replace('/', '').replace('?', '').replace('*', '').replace('[', '').replace(']', '')
    worksheet.title = valid_title
    worksheet.cell(row=1, column=1, value="Name")
    worksheet.cell(row=1, column=2, value="UserID")
    worksheet.cell(row=1, column=3, value="Date")
    worksheet.cell(row=1, column=4, value="Ratings")
    worksheet.cell(row=1, column=5, value="Comments")

    total_reviews = int(input("How many reviews?:\n"))

    save_content(app_id, workbook, worksheet, total_reviews)

    workbook.save(f"{valid_title}_Reviews.xlsx")

if __name__ == '__main__':
    main()

APP's name:
Day One Journal: Private Diary
Name: Day One Journal: Private Diary, ID: 1044867788
Name: 1 Second Everyday Diary, ID: 587823548
Name: Tinybeans Private Family Album, ID: 521633042
Name: Diarly: Diary, Private Journal, ID: 1387167765
Name: Diary-Journal with lock, ID: 1590102229
Name: Happyfeed: Video Diary Journal, ID: 707073943
Name: Diary With Password, ID: 1252208166
Name: May Diary - Gratitude Journal, ID: 658236285
Name: My Diary - Private Journal, ID: 6478656385
Name: Diary - Private Note With Lock, ID: 1454490013
Name: My Wonderful Days Journal, ID: 434356065
Name: Personal Diary (Journal) App, ID: 1052570257
Name: OneDay - Diary Journal, ID: 1578156802
Name: Diary, Journal, Notes - Diaro, ID: 882519460
Name: GoJournal: Diary & Planner, ID: 1596865689
Name: Video Dairy: Daily Journal, ID: 1483531474
Name: Secret Diary-Gratitude Journal, ID: 1107242504
Name: Simple Diary - Daily Journal, ID: 1388195923
Name: Diary Journal - With Lock, ID: 1572225395
Name: Orange Diar